# Without LoRA

In [5]:
from fastai.vision.all import *

path = untar_data(URLs.PETS)/'images'
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: x[0].isupper(), item_tfms=Resize(224))

learn = vision_learner(dls, "resnet50", metrics=[error_rate, accuracy])

def count_parameters(model):
    trainable_params = 0
    non_trainable_params = 0
    
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
        else:
            non_trainable_params += param.numel()
    
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Non-trainable parameters: {non_trainable_params:,}")
    print(f"Total parameters: {trainable_params + non_trainable_params:,}")
    
    return trainable_params, non_trainable_params

count_parameters(learn.model)


# for name, param in learn.model.named_parameters():
#     print(f"{name}: requires_grad = {param.requires_grad}")

learn.summary()

Trainable parameters: 2,160,512
Non-trainable parameters: 23,454,912
Total parameters: 25,615,424


Sequential (Input shape: 64 x 3 x 224 x 224)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 64 x 112 x 112 
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     64 x 64 x 56 x 56   
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
Identity                                                       
ReLU                                                           
Identity                  

In [6]:
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.187871,0.093667,0.016238,0.983762,00:16
1,0.075238,0.138535,0.010825,0.989175,00:15
2,0.038429,0.239099,0.013532,0.986468,00:15


## With LoRA

In [10]:
from fastai.vision.all import *
from lora_adapters import LoraConv2d, apply_adapter, mark_only_lora_as_trainable

path = untar_data(URLs.PETS)/'images'
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: x[0].isupper(), item_tfms=Resize(224))

learn = vision_learner(dls, "resnet50", metrics=[error_rate, accuracy])

# freeze all layers
# for param in learn.model.parameters():
#     param.requires_grad = False

# learn.model[0].model = apply_adapter(learn.model[0].model, LoraConv2d, rank=16)
learn.model[0].model = mark_only_lora_as_trainable(learn.model[0].model)


def count_parameters(model):
    trainable_params = 0
    non_trainable_params = 0
    
    for param in model.parameters():
        if param.requires_grad:
            trainable_params += param.numel()
        else:
            non_trainable_params += param.numel()
    
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Non-trainable parameters: {non_trainable_params:,}")
    print(f"Total parameters: {trainable_params + non_trainable_params:,}")
    
    return trainable_params, non_trainable_params


count_parameters(learn.model)

for name, param in learn.model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

learn.summary()

Trainable parameters: 2,107,392
Non-trainable parameters: 23,508,032
Total parameters: 25,615,424
0.model.conv1.weight: requires_grad = False
0.model.bn1.weight: requires_grad = False
0.model.bn1.bias: requires_grad = False
0.model.layer1.0.conv1.weight: requires_grad = False
0.model.layer1.0.bn1.weight: requires_grad = False
0.model.layer1.0.bn1.bias: requires_grad = False
0.model.layer1.0.conv2.weight: requires_grad = False
0.model.layer1.0.bn2.weight: requires_grad = False
0.model.layer1.0.bn2.bias: requires_grad = False
0.model.layer1.0.conv3.weight: requires_grad = False
0.model.layer1.0.bn3.weight: requires_grad = False
0.model.layer1.0.bn3.bias: requires_grad = False
0.model.layer1.0.downsample.0.weight: requires_grad = False
0.model.layer1.0.downsample.1.weight: requires_grad = False
0.model.layer1.0.downsample.1.bias: requires_grad = False
0.model.layer1.1.conv1.weight: requires_grad = False
0.model.layer1.1.bn1.weight: requires_grad = False
0.model.layer1.1.bn1.bias: requires

Sequential (Input shape: 64 x 3 x 224 x 224)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 64 x 112 x 112 
Conv2d                                    9408       False     
BatchNorm2d                               128        False     
ReLU                                                           
____________________________________________________________________________
                     64 x 64 x 56 x 56   
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        False     
ReLU                                                           
Conv2d                                    36864      False     
BatchNorm2d                               128        False     
Identity                                                       
ReLU                                                           
Identity                  

In [8]:
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.414196,1.079915,0.512179,0.487821,00:20
1,1.350015,1.129787,0.515562,0.484438,00:18
2,1.349205,1.100330,0.520974,0.479026,00:18


In [ ]:
import timm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('resnet50', pretrained=True).to(device)
apply_adapter(model, LoraConv2d, rank=16)

trainable_layers = []
for name, param in model.named_parameters():

    trainable_layers.append((name, param.numel(), param.requires_grad))

    print(f"Layer: {name}")
    print(f"  Trainable: {param.requires_grad}")
    print(f"  Shape: {param.shape}")
    print("---")

In [ ]:
trainable_layers